In [ ]:
import ipywidgets as widgets
import ipyvue as vue
from traitlets import (Dict, Unicode, List, Any)

In [ ]:
mydiv = vue.Html(tag='div', children=['mydiv'])

In [ ]:
class CoolComponent(vue.VueTemplate):
    input = Unicode().tag(sync=True)
    something = Unicode('defaultvalue').tag(sync=True)
    template = Unicode('''
        <div style="border: 1px solid green; margin: 2px" @click="append_one()">
            [{{ something }}] input: {{ input }} 
        </div>
    ''').tag(sync=True)
    
    def vue_append_one(self, *args):
        self.input = f'{self.input}1'
    
CoolComponent(input='some text')    

In [ ]:
class Cls(vue.VueTemplate):
    texts = List(['xxxx', 'yyyy']).tag(sync=True)
    direct = Unicode('aaa').tag(sync=True)
    template = Unicode('''
        <div>
            <div style="border: 1px solid black">
               <cool-component v-for="t in texts" :input="t" :key="t" />
            </div>
            <cool-component :input="direct" something="---"/>
            <w-button v-for="t in texts" :description="t" :key="t" />
            <w-button description="no v-for"/>
            <some-instance />
        </div>
    ''').tag(sync=True)

cls = Cls(components={
    'cool-component': CoolComponent,
    'some-instance': mydiv,
    'w-button': widgets.Button})   

cls

In [ ]:
cls.texts=['xxxx', 'zzzz']

In [ ]:
cls._component_instances

In [ ]:
cls.direct = 'bbb'

# Non serializable properties

In [ ]:
class ImNotSerializable():
    def __init__(self, param):
        self.param = param

class NoSerComponent(vue.VueTemplate):
    backend_only = Any()
    input = Unicode('-').tag(sync=True)
    template = Unicode('''
        <div style="border: 1px solid green; margin: 2px" @click="append_from_backend()">
            {{ input }}
        </div>
    ''').tag(sync=True)
    
    def vue_append_from_backend(self, *args):
        self.input = f'{self.input} {self.backend_only.param}'
        

In [ ]:
class Cls2(vue.VueTemplate):
    my_backend_only = Any().tag(sync_ref=True)
    my_backend_only_collection = Any().tag(sync_ref=True)

    template = Unicode('''
        <div>
            <p>Click on component to see content of backend object:</p>
            
            <no-ser input="value" :backend_only="my_backend_only" />
            
            <no-ser input="collection" :backend_only="my_backend_only_collection['key1'][0]" />
            
            <no-ser input="function" :backend_only="{PY_FN: 'create_backend_only', args: ['with fn']}" />
        </div>
    ''').tag(sync=True)
    
    components = Dict({
        'no-ser': NoSerComponent
    }).tag(sync=True, **vue.VueTemplate.class_component_serialization)
   
    def create_backend_only(self, param):
        return ImNotSerializable(param)

cls2 = Cls2(
    my_backend_only=ImNotSerializable('my_backend_only'),
    my_backend_only_collection={'key1': [ImNotSerializable('my_backend_only_collection["key1"][0]')]},
)   

cls2

In [ ]:
cls2.my_backend_only = ImNotSerializable('value changed')
# click on first component again to see the change

In [ ]:
cls2.my_backend_only_collection = {'key1': [ImNotSerializable('collection changed')]}
# click on second component again to see the change